In [20]:
import pandas as pd
from kafka import KafkaProducer
from json import dumps
from time import sleep
import json
from config import ip
import requests
from datetime import datetime

In [2]:
producer = KafkaProducer(bootstrap_servers=[f'{ip}:9092'],
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))

In [6]:
producer.send('Price', value={'Lee': 'Simpson'})

In [17]:
url = "https://api.coinbase.com/v2/exchange-rates?currency=BTC"
response = requests.get(url)
ex_rates = json.loads(response.text)
ex_rates['data']['rates']['USD']

'31235.3449999544273037'

In [28]:
while True:


    dt = datetime.now()
    dt.strftime("%Y-%m-%d %H:%M:%S")

    url = "https://api.exchange.coinbase.com/products/TRAC-USD/ticker"
    response = requests.get(url)
    TRAC = json.loads(response.text)
    TRAC['time'] = dt.strftime("%Y-%m-%d %H:%M:%S")
    producer.send('Price', value=TRAC)
    sleep(15)



KeyboardInterrupt: 

In [ ]:
producer.flush()